# Visualize the 3-30-300 rule on a map. 

[![colab](https://colab.research.google.com/assets/colab-badge.svg)]() [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)]()

**Author**: Willeke A'Campo

**Description:** This notebooks shows the results of the 3-30-300 rule on a map for Kristiansand Municipality in Norway. 



### Map the indicators for the 3-30-300-rule

In [ ]:
import os
import leafmap
import pandas as pd

In [ ]:
municipality = "kristiansand"
src_crs = "EPSG:25832"
project_dir = os.getcwd()

input_file = os.path.join(project_dir, "data", municipality, "processed", f"{municipality}_district_3-30-300_stat.parquet" )
# add layers to gdf for mapping
gdf_mapping = leafmap.read_parquet(
    input_file, 
    return_type='gdf', 
    src_crs= src_crs , 
    dst_crs="EPSG:4326"
    )

# Calculate the center of the study_area GeoDataFrame
center = gdf_mapping.geometry.unary_union.centroid

display(gdf_mapping[[
    "kommunenavn","grunnkretsnummer", "grunnkretsnavn", 
    "n_trees", "n_bldg", "n_res_bldg", 
    "n_trees_near_rbldg","perc_near_rbldg","labels_near_rbldg",
    "a_crown", "perc_crown", "labels_perc_crown",
    "n_res_bldg_near_gs","perc_near_gs", "labels_near_gs",
    "a_open_space", "a_private_space", "a_public_space"
    ]])

**Rule 3 | all residents should be able to see at least 3 trees from their home.**

In [ ]:
m = leafmap.Map()
m.set_center(center.x, center.y, zoom=13)
m.add_basemap("CartoDB.Positron")
m.add_data(
    gdf_mapping, column='perc_near_rbldg', scheme='Quantiles', cmap='Greens', legend_title='Number of Trees near Residential Buildings'
)
m

**Rule 30: all neighbourhoods should have 30% tree canopy cover.**

In [ ]:
m = leafmap.Map()
m.set_center(center.x, center.y, zoom=13)
m.add_basemap("CartoDB.Positron")
m.add_data(
    gdf_mapping, column='perc_crown', scheme='Quantiles', cmap='Greens', legend_title='Crown Area (%)'
)
m

**Rule 300:  all residents should be able to access a green area within 300 m from their home.**

In [ ]:
gdf_mapping_gs = gdf_mapping.dropna(subset=['n_res_bldg_near_gs_norm'])
m = leafmap.Map()
m.set_center(center.x, center.y, zoom=13)
m.add_basemap("CartoDB.Positron")
m.add_data(
    gdf_mapping_gs, column='perc_near_gs', scheme='Quantiles', cmap='Greens', legend_title='Number of Residential Buildings near green spaces'
)
m